<a href="https://colab.research.google.com/github/saintblue97/DeepNN_Optiver/blob/main/brett_10_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive; drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
#!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v optiver-trading-at-the-close        # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
# !kaggle competitions leaderboard --show                       # print public leaderboard

- competition is now set to: optiver-trading-at-the-close
100% 201M/201M [00:06<00:00, 32.4MB/s]


In [4]:
%%time
%%capture
%reset -f
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, os, tqdm, re, sys, cv2, skimage, xgboost, lightgbm as lgb, librosa

ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=300): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 2.16 s, sys: 431 ms, total: 2.59 s
Wall time: 4.88 s


In [5]:
def is_google_colab():
    """Check if the environment is Google Colab.

    Returns:
        bool: True if in Google Colab, False otherwise.
    """
    try:
        import google.colab
        return True
    except ImportError:
      return False

if is_google_colab():
  file_path = ''
else:
  file_path = '/kaggle/input/optiver-trading-at-the-close/'

In [6]:
sample_submission_path = file_path + 'example_test_files/sample_submission.csv'

sample_submission = pd.read_csv(sample_submission_path); sample_submission

,time_id,row_id,target
0,26290,478_0_0,0
1,26290,478_0_1,0
...,...,...,...
32998,26454,480_540_198,0
32999,26454,480_540_199,0


# Load Data

In [8]:
tmr = Timer()
RUN_CV = True

⏳ started. You have 300 sec. Good luck!


In [9]:
train_data_path = file_path + 'train.csv'
df = pd.read_csv(train_data_path);
df = df.dropna(subset=['target']); df

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3.18e+06,1,1.0,1.34e+07,NaN,NaN,1.0,60651.50,1.0,8493.03,1.0,-3.03,0,0_0_0
1,1,0,0,1.67e+05,-1,1.0,1.64e+06,NaN,NaN,1.0,3233.04,1.0,20605.09,1.0,-5.52,0,0_0_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237978,198,480,540,1.00e+06,1,1.0,9.48e+07,1.0,1.0,1.0,125631.72,1.0,669893.00,1.0,-1.54,26454,480_540_198
5237979,199,480,540,1.88e+06,-1,1.0,2.41e+07,1.0,1.0,1.0,250081.44,1.0,300167.56,1.0,-6.53,26454,480_540_199


In [10]:
test_data_path = file_path + 'example_test_files/test.csv'
df_test  = pd.read_csv(test_data_path); df_test

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,time_id,row_id,currently_scored
0,0,478,0,3.75e+06,-1,1.0,1.15e+07,NaN,NaN,1.0,22940.00,1.0,9177.60,1.0,26290,478_0_0,False
1,1,478,0,9.86e+05,-1,1.0,3.85e+06,NaN,NaN,1.0,1967.90,1.0,19692.00,1.0,26290,478_0_1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32998,198,480,540,1.00e+06,1,1.0,9.48e+07,1.0,1.0,1.0,125631.72,1.0,669893.00,1.0,26454,480_540_198,False
32999,199,480,540,1.88e+06,-1,1.0,2.41e+07,1.0,1.0,1.0,250081.44,1.0,300167.56,1.0,26454,480_540_199,False


In [11]:
print('Shape of training data =', df.shape)
print('Shape of testing data  =', df_test.shape)

Shape of training data = (5237892, 17)
Shape of testing data  = (33000, 17)


In [13]:
if is_google_colab():
  !pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.7 MB/s eta 0:00:00


In [14]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler, RobustScaler,OneHotEncoder
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import ARDRegression, BayesianRidge, LinearRegression, Ridge, Lars
from itertools import combinations
import gc
import os
import time
from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool
from lightgbm import LGBMRegressor
from sklearn.pipeline import Pipeline

In [15]:
# n_val = 50000

tX, tY = df.drop('target', axis=1), df.target    # full training set

X_train = tX
y_train = tY

# from sklearn.model_selection import train_test_split
# X_train, X_valid, y_train, y_valid = train_test_split(tX, tY, test_size=0.1)
# X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1)

# Preprocessing

In [16]:
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop):
        self.columns_to_drop = columns_to_drop

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Only drop columns that exist in X
        columns_to_drop = [col for col in self.columns_to_drop if col in X.columns]
        return X.drop(columns_to_drop, axis=1)

class DataFrameSimpleImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='mean', fill_value=None):
        self.strategy_ = strategy
        self.fill_value_ = fill_value

    def fit(self, X, y=None):
        if self.strategy_ == 'mean':
            self.fill_values_ = X.mean()
        elif self.strategy_ == 'median':
            self.fill_values_ = X.median()
        elif self.strategy_ == 'constant':
            if self.fill_value_ is None:
                raise ValueError("fill_value must be provided for strategy='constant'")
            self.fill_values_ = pd.Series(self.fill_value_, index=X.columns)
        else:
            raise ValueError(f"Unknown strategy type: {self.strategy_}")

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill_values_)

class MovingAverageTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, window_size, columns):
        self.window_size = window_size  # The size of the window for moving average
        self.columns = columns          # The columns on which to apply moving average

    def fit(self, X, y=None):
        return self  # No fitting process, so just return self

    def calculate_moving_average(self, series):
        return series.rolling(window=self.window_size).mean()

    def transform(self, X):
        X_new = X.copy()

        for col in self.columns:
            X_new[f'{col}_sma_{self.window_size}'] = X_new.groupby('stock_id')[col].transform(self.calculate_moving_average)

            X_new[f'{col}_sma_{self.window_size}'].fillna(X_new[col], inplace=True)


        return X_new


In [17]:
# generate imbalance features
import numpy as np
import pandas as pd
from itertools import combinations

def imbalance_features(df):
    # Create a copy of the DataFrame to avoid modifying the original data
    df_copy = df.copy()

    # Define price and size columns
    prices = ["reference_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size"]

    # Calculate the total volume for each entry
    # Volume is the sum of ask and bid sizes
    df_copy["volume"] = df_copy["ask_size"] + df_copy["bid_size"]

    # Calculate the mid price for each entry
    # Mid price is the average of ask and bid prices
    df_copy["mid_price"] = (df_copy["ask_price"] + df_copy["bid_price"]) / 2

    # Calculate liquidity imbalance
    # Liquidity imbalance is the difference between bid and ask sizes divided by their sum
    df_copy["liquidity_imbalance"] = (df_copy["bid_size"] - df_copy["ask_size"]) / (df_copy["bid_size"] + df_copy["ask_size"])

    # Calculate price imbalances for all combinations of price types
    for price_pair in combinations(prices, 2):
        col_name = f"{price_pair[0]}_{price_pair[1]}_imb"
        df_copy[col_name] = (df_copy[price_pair[0]] - df_copy[price_pair[1]]) / (df_copy[price_pair[0]] + df_copy[price_pair[1]])

    # Calculate the price spread
    # Price spread is the difference between ask and bid prices
    df_copy["price_spread"] = df_copy["ask_price"] - df_copy["bid_price"]

    # Calculate spread depth ratio
    # Spread depth ratio is the price spread divided by the sum of bid and ask sizes
    df_copy["spread_depth_ratio"] = df_copy["price_spread"] / (df_copy["bid_size"] + df_copy["ask_size"])

    # Calculate aggregate statistics (mean and standard deviation) for prices and sizes
    df_copy["all_prices_mean"] = df_copy[prices].mean(axis=1)
    df_copy["all_prices_std"] = df_copy[prices].std(axis=1)
    df_copy["all_sizes_mean"] = df_copy[sizes].mean(axis=1)
    df_copy["all_sizes_std"] = df_copy[sizes].std(axis=1)

    # Return the modified DataFrame with new features
    return df_copy


def other_features(df):
    # Create a copy of the DataFrame to avoid modifying the original data
    df_copy = df.copy()

    # Calculate the day of the week from 'date_id'
    # 'dow' represents the trading day of the week, which can capture weekly cyclic patterns
    df_copy["dow"] = df_copy["date_id"] % 5

    # Calculate the day of the month from 'date_id'
    # 'dom' represents the trading day of the month, which can capture monthly cyclic patterns
    df_copy["dom"] = df_copy["date_id"] % 20

    # Extract seconds within the minute from 'seconds_in_bucket'
    # 'seconds' captures the precise moment within a minute, useful for high-frequency data analysis
    df_copy["seconds"] = df_copy["seconds_in_bucket"] % 60

    # Calculate the minute within an hour from 'seconds_in_bucket'
    # 'minute' identifies the minute within an hour, helping to understand intra-hour patterns
    df_copy["minute"] = df_copy["seconds_in_bucket"] // 60

    # Return the modified DataFrame with new features
    return df_copy


def generate_all_features(df):
    # Create a copy of the DataFrame to avoid modifying the original data
    df_copy = df.copy()

    # Filter out specific columns ('row_id', 'time_id', 'target') from the DataFrame
    # This step focuses on retaining only the features that are relevant for further processing
    cols = [c for c in df_copy.columns if c not in ["row_id", "time_id", "target"]]
    df_copy = df_copy[cols]

    # Apply the 'imbalance_features' function to add new features related to market imbalances
    df_copy = imbalance_features(df_copy)

    # Apply the 'other_features' function to add new temporal and stock-specific features
    df_copy = other_features(df_copy)

    # Free up memory - useful when working with large datasets
    gc.collect()

    # Filter out specific columns ('row_id', 'target', 'time_id', 'date_id') to finalize the feature set
    feature_name = [i for i in df_copy.columns if i not in ["row_id", "target", "time_id", "date_id"]]

    # Return the DataFrame with the generated feature set
    return df_copy[feature_name]

In [18]:
class ImbalanceFeaturesTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        transformed_X = imbalance_features(X)
        return transformed_X


class OtherFeaturesTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self  # No fitting process needed

    def transform(self, X):
        return other_features(X)


class AllFeaturesTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self  # No fitting process needed

    def transform(self, X):
        return generate_all_features(X)


In [21]:
from sklearn.compose import ColumnTransformer

# Set up preprocessing pipeline
preprocess_steps = [
    ('drop_columns', DropColumns(['time_id', 'currently_scored'])),
    ('imputer1', DataFrameSimpleImputer(strategy='constant', fill_value=0)),
    ('moving_average', MovingAverageTransformer(window_size=2, columns=['ask_size', 'ask_price', 'bid_price', 'ask_size', 'reference_price', 'wap'])),
    ('imbalance_features', ImbalanceFeaturesTransformer()),
    ('other_features', OtherFeaturesTransformer()),
    ('all_features', AllFeaturesTransformer()),
    ('imputer2', DataFrameSimpleImputer(strategy='constant', fill_value=0)),
]
pipeline = Pipeline(preprocess_steps)

In [22]:
X_train_pp = pipeline.fit_transform(X_train)
X_train_pp

,stock_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,...,price_spread,spread_depth_ratio,all_prices_mean,all_prices_std,all_sizes_mean,all_sizes_std,dow,dom,seconds,minute
0,0,0,3.18e+06,1,1.0,1.34e+07,0.0,0.0,1.0,60651.50,...,2.14e-04,3.09e-09,1.0,1.17e-04,4.48e+06,7.71e+06,0,0,0,0
1,1,0,1.67e+05,-1,1.0,1.64e+06,0.0,0.0,1.0,3233.04,...,7.64e-04,3.20e-08,1.0,3.68e-04,5.55e+05,9.41e+05,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237978,198,540,1.00e+06,1,1.0,9.48e+07,1.0,1.0,1.0,125631.72,...,2.40e-04,3.02e-10,1.0,1.29e-04,3.19e+07,5.45e+07,0,0,0,9
5237979,199,540,1.88e+06,-1,1.0,2.41e+07,1.0,1.0,1.0,250081.44,...,3.18e-04,5.78e-10,1.0,1.51e-04,8.21e+06,1.37e+07,0,0,0,9


# Make submission

In [23]:
from sklearn.model_selection import train_test_split
X_train_pp, X_valid_pp, y_train_pp, y_valid_pp = train_test_split(X_train_pp, y_train,
                                                         random_state=0, test_size=0.01)
X_train_pp, X_test_pp, y_train_pp, y_test_pp = train_test_split(X_train_pp, y_train_pp,
                                                       random_state=0, test_size=0.01)

In [24]:
%%time
# Train model
num_iter = 3500
ctb_params = dict(iterations=num_iter,
                  learning_rate=0.25,
                  depth=8,
                  l2_leaf_reg=30,
                  bootstrap_type='Poisson',
                  subsample=0.66,
                  loss_function='MAE',
                  eval_metric = 'MAE',
                  metric_period=num_iter//10,
                  od_type='Iter',
                  od_wait=30,
                  allow_writing_files=False,
                  random_state=0,
                  task_type='GPU'
                 )
models = {
    #'lgb': LGBMRegressor(objective='regression_l1', n_estimators=500),
    #'xgb': XGBRegressor(tree_method='hist', objective='reg:absoluteerror', n_estimators=250),
    'catboost': CatBoostRegressor(**ctb_params)
}
for name, model in models.items():
    print(f'Training {name} model')

    model.fit(
        X_train_pp,
        y_train_pp.values.ravel(),
        eval_set=Pool(X_valid_pp, y_valid_pp)
    )
    if RUN_CV:
        print("Initiating Cross Validation and Computing MAE...")
        # Perform cross-validation and calculate MAE
        scores = cross_val_score(model, X_train_pp, y_train_pp, scoring='neg_mean_absolute_error', cv=3)
        mae = -scores.mean()
        print(f'{name} model MAE (CV): {mae:.4f}')

    # Calculate MAE for Test Dataset
    print("Computing MAE...")
    mae = mean_absolute_error(y_test_pp, model.predict(X_test_pp))
    print(f'{name} model MAE (Test dataset): {mae:.4f}')
    print(model.get_best_iteration())

Training catboost model
0:	learn: 6.4057536	test: 6.4157672	best: 6.4157672 (0)	total: 107ms	remaining: 6m 14s
350:	learn: 6.2858395	test: 6.2933690	best: 6.2933690 (350)	total: 31.5s	remaining: 4m 42s
700:	learn: 6.2717953	test: 6.2813186	best: 6.2813186 (700)	total: 1m 2s	remaining: 4m 11s
1050:	learn: 6.2631126	test: 6.2744504	best: 6.2744504 (1050)	total: 1m 29s	remaining: 3m 28s
1400:	learn: 6.2565446	test: 6.2697079	best: 6.2697079 (1400)	total: 1m 58s	remaining: 2m 57s
1750:	learn: 6.2509614	test: 6.2659660	best: 6.2659660 (1750)	total: 2m 27s	remaining: 2m 27s
2100:	learn: 6.2459385	test: 6.2625659	best: 6.2625659 (2100)	total: 2m 55s	remaining: 1m 56s
2450:	learn: 6.2413344	test: 6.2597314	best: 6.2597314 (2450)	total: 3m 22s	remaining: 1m 26s
2800:	learn: 6.2370104	test: 6.2570997	best: 6.2570997 (2800)	total: 3m 51s	remaining: 57.8s
3150:	learn: 6.2329934	test: 6.2548785	best: 6.2548785 (3150)	total: 4m 20s	remaining: 28.8s
3499:	learn: 6.2290659	test: 6.2526913	best: 6.2526

In [25]:
# if not is_google_colab():
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [26]:
from sklearn.metrics import mean_absolute_error
import pandas as pd

if is_google_colab():

    # Lists to store true and predicted targets
    true_targets = []
    predicted_targets = []

    for (test, revealed_targets, sample_prediction) in iter_test:
        # Predict target values
        test_processed = pipeline.transform(test)
        predictions = np.mean([model.predict(test_processed) for _, model in models.items()], 0)

        # Only take as many predictions and true targets as the minimum of the two
        min_len = min(len(predictions), len(revealed_targets))

        # Trim down the predictions list so that it is the same size as the revealed targets
        current_predictions = predictions[:min_len]
        current_true_targets = revealed_targets['revealed_target'].values[:min_len]

        # Extend the lists of true targets and predictions with the ones from the current batch
        true_targets.extend(current_true_targets)
        predicted_targets.extend(current_predictions)

        # Make the actual prediction and add to sample_prediction df
        sample_prediction['target'] = predictions

        # Submit the predictions for this chunk of test data
        env.predict(sample_prediction)

    # Convert lists to dataframes
    df_true = pd.DataFrame(true_targets, columns=['true_target'])
    df_pred = pd.DataFrame(predicted_targets, columns=['predicted_target'])

    # Calculate and print the MAE
    mae = mean_absolute_error(df_true.fillna(0), df_pred)
    print("Mean Absolute Error:", mae)
    # 5.261823197997517

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
Mean Absolute Error: 5.650593995733539


In [27]:
if not is_google_colab():
  counter = 0
  for (test, revealed_targets, sample_prediction) in iter_test:
      test_processed = pipeline.transform(test).values
      sample_prediction['target'] = np.mean([model.predict(test_processed) for _, model in models.items()], 0)
      env.predict(sample_prediction)
      counter += 1

In [28]:
tmr.ShowTime()    # measure Colab's runtime.

Runtime is 1246 sec > 300 sec limit!!!


The **preprocessing** pipeline for this assignment has these steps:
1. Data Loading and Initial Setup:
  -Load training and test data from CSV files.
  -Remove rows with missing target values from the training dataset.
  -Print the shapes of the training and test datasets to verify their sizes.

2. Data Splitting for Model Training and Validation:
  -Split the full training dataset (df) into features (tX) and the target variable (tY).

3. Preprocessing Steps Defined:
  -Drop Unnecessary Columns: Use DropColumns to remove specified columns from the data.
  -Data Imputation: Apply DataFrameSimpleImputer to fill missing values with  the value 0.
  -Moving Average Transformation: Implement MovingAverageTransformer to calculate the moving average for selected columns, grouped by stock ID.
  -Imbalance Feature Generation: Use ImbalanceFeaturesTransformer to create new features related to market imbalances.
  -Temporal and Stock-Specific Features: Apply OtherFeaturesTransformer to add temporal features (like day of the week) and stock-specific features.
  -All Features Assembly: Utilize AllFeaturesTransformer to compile all the generated features into a comprehensive feature set.
  -Final Data Imputation: Apply another round of imputation to ensure no missing values in the final dataset.

4. Pipeline Assembly and Application:
  -Assemble the above preprocessing steps into a Pipeline.
  -Apply this pipeline to transform the X_train dataset into X_transformed, ready for model training.

### **Why did you choose these elements?**
**Data Preprocessing and Feature Engineering** \
The first step in the preprocessing pipeline was to load in the dataset, remove all training data that has an NAN value and use all the training data for the pipeline.

Then the next step was to remove uneccesary features that are not needed for the model and umpute any NAN values in the dataset with 0 as this created the best performing pipeline.

Then the moving average was added to the preprocessing pipeline which adds valuable insights into market trends and behaviors, which are vital for any analysis or predictive modeling in the financial sector. Moving averages smooth out short-term fluctuations and highlight longer-term trends in data, which is crucial for identifying general market directions in financial time series. By averaging data over a period, this method helps reduce the noise and allows for a clearer analysis of the underlying trends in the market. The grouping by stock_id before applying the moving average allows for tailored analysis for each stock, recognizing that different stocks may exhibit different trading patterns. The ability to apply this transformation to various columns (like prices and sizes) makes it a versatile tool for exploring different aspects of market behavior.

Then the follow features were created:

Volume: The volume feature is the sum of ask_size and bid_size, representing the total trading size for each stock, which can indicate the overall trading activity and liquidity for that stock.

Mid Price: The mid_price feature is the average of ask_price and bid_price, providing a central reference point for the stock's price, which can be a more stable indicator of the stock's value compared to individual bid or ask prices.

Liquidity Imbalance: The liquidity_imbalance feature calculates the difference between bid_size and ask_size divided by their sum, indicating whether there is more demand (buying pressure) or supply (selling pressure) in the market, which can signal potential price movements.

Price Imbalances for Combinations of Prices: These features, calculated for every pair of price types (e.g., reference_price_ask_price_imb), measure the relative differences between various price metrics, offering insights into discrepancies or anomalies in pricing that could signal trading opportunities or market inefficiencies.

Price Spread: The price_spread feature, calculated as the difference between ask_price and bid_price, indicates the gap between what buyers are willing to pay and what sellers are asking, a direct measure of market liquidity and volatility.

Spread Depth Ratio: The spread_depth_ratio is the price spread divided by the sum of bid_size and ask_size, offering a normalized measure of the spread relative to the market depth, which can help in understanding how significant the price spread is in the context of current market activity.

Aggregate Statistics for Prices and Sizes: The features all_prices_mean, all_prices_std, all_sizes_mean, and all_sizes_std provide summary statistics (mean and standard deviation) for prices and sizes, giving a broader view of the market conditions by summarizing key aspects of the price and size distributions, useful for capturing general market trends and volatility.

Day of the Week (dow): The dow feature is calculated from date_id and represents the trading day of the week, which can be crucial in capturing weekly cyclic patterns in the stock market, such as increased trading activity on certain days.

Day of the Month (dom): The dom feature, derived from date_id, indicates the trading day of the month and is useful for identifying monthly patterns or trends, such as end-of-month financial reporting effects.

Seconds Within a Minute (seconds): The seconds feature is the remainder of seconds_in_bucket divided by 60, pinpointing the exact second within a minute, which is particularly relevant for high-frequency trading data where precision to the second can be significant.

Minute of the Hour (minute): The minute feature, calculated as the quotient of seconds_in_bucket divided by 60, marks the minute within an hour, aiding in understanding intra-hour trading patterns and behaviors.

The final step in the preprocessing pipeline was to go back through the training features and impute any NAN value with 0 to make sure all features can be inputted to the model fit

### **What else did you try that worked or didn't?**
Imputing values with the mean or median did perform as well as imputing with 0.

We were unsuccessful in implementing lag features to our preprocessing pipeline largly because the submission limitations of the kaggle assignment. The competition submission was structured in a way that we needed to call "iter_test" from their environment a specified number of times and any significant manipulation with data from multiple iterations cause the submission to fail so we could not implement these features.

Polynominal Features and Standard Scalar did not improve the model performance and are less effective in a time series application.